In [1]:
import numpy as np
from collections import namedtuple
CentersDictionary = namedtuple('CentersDictionary', ('idx', 'X', 'probs', 'lam', 'qbar'))

def __load_gpu_module(force_cpu: bool):
    xp = np
    xp.asnumpy = np.asarray
    if not force_cpu:
        try:
            import cupy as cp
            xp = cp
        except ImportError:
            print("cupy not found, defaulting to numpy")
    return xp


def __get_progress_bar(total=-1, disable=False):
    """Helper function to get a tqdm progress bar (or a simple fallback otherwise)"""
    class ProgBar(object):
        def __init__(self, total=-1, disable=False):
            self.disable = disable
            self.t = 0
            self.total = total
            self.debug_string = ""

        def __enter__(self):
            return self

        def __exit__(self, *args, **kwargs):
            pass

        def set_postfix(self, **kwargs):
            self.debug_string = ""
            for arg in kwargs:
                self.debug_string += "{}={} ".format(arg, kwargs[arg])

        def update(self):
            if not self.disable:
                self.t += 1
                print_str = "{}".format(self.t)

                if self.total > 0:
                    print_str += "/{}".format(self.total)

                print_str += ": {}".format(self.debug_string)

                if len(print_str) < 80:
                    print_str = print_str + " "*(80 - len(print_str))
                print(print_str, end='\r', flush=True)

            if self.t == self.total:
                print("")
    try:
        from tqdm import tqdm
        progress_bar = tqdm(total=total, disable=disable)
    except ImportError:
        progress_bar = ProgBar(total=total, disable=disable)
    return progress_bar


def __stable_invert_root(U: np.ndarray, S: np.ndarray):
    n = U.shape[0]
    assert U.shape == (n, n)
    assert S.shape == (n,)
    # threshold formula taken from pinv2's implementation of numpy/scipy
    thresh = S.max() * max(S.shape) * np.finfo(S.dtype).eps
    stable_eig = np.logical_not(np.isclose(S, 0., atol=thresh))
    m = sum(stable_eig)
    U_thin = U[:, stable_eig]
    S_thin = S[stable_eig]
    assert U_thin.shape == (n, m)
    assert S_thin.shape == (m,)
    S_thin_inv_root = (1 / np.sqrt(S_thin)).reshape(-1, 1)
    return U_thin, S_thin_inv_root


def compute_tau(centers_dict: CentersDictionary,
                X: np.ndarray,
                similarity_func: callable,
                lam_new: float,
                force_cpu=False):
    xp = __load_gpu_module(force_cpu)
    diag_norm = np.asarray(similarity_func.diag(X))
    # (m x n) kernel matrix between samples in dictionary and dataset X
    K_DU = xp.asarray(similarity_func(centers_dict.X, X))
    # the estimator proposed in Calandriello et al. 2017 is
    # diag(XX' - XX'S(SX'XS + lam*I)^(-1)SXX')/lam
    # here for efficiency we collect an S inside the inverse and compute
    # diag(XX' - XX'(X'X + lam*S^(-2))^(-1)XX')/lam
    # note that in the second term we take care of dropping the rows/columns of X associated
    # with 0 entries in S
    U_DD, S_DD, _ = np.linalg.svd(xp.asnumpy(similarity_func(centers_dict.X, centers_dict.X)
                                             + lam_new * np.diag(centers_dict.probs)))
    U_DD, S_root_inv_DD = __stable_invert_root(U_DD, S_DD)
    E = xp.asarray(S_root_inv_DD * U_DD.T)
    # compute (X'X + lam*S^(-2))^(-1/2)XX'
    X_precond = E.dot(K_DU)
    # the diagonal entries of XX'(X'X + lam*S^(-2))^(-1)XX' are just the squared
    # ell-2 norm of the columns of (X'X + lam*S^(-2))^(-1/2)XX'
    tau = (diag_norm - xp.asnumpy(xp.square(X_precond, out=X_precond).sum(axis=0))) / lam_new
    assert np.all(tau >= 0.), ('Some estimated RLS is negative, this should never happen. '
                               'Min prob: {:.5f}'.format(np.min(tau)))
    return tau

def reduce_lambda(X: np.ndarray,
                  similarity_func: callable,
                  centers_dict: CentersDictionary,
                  lam_new: float,
                  random_state: np.random.RandomState,
                  qbar=None,
                  force_cpu=False):
    n, d = X.shape
    if qbar is None:
        qbar = centers_dict.qbar
    red_ratio = centers_dict.lam / lam_new
    assert red_ratio >= 1.
    diag = np.asarray(similarity_func.diag(X))
    # compute upper confidence bound on RLS of each sample, overestimate (oversample) by a qbar factor
    # to boost success probability at the expenses of a larger sample (dictionary)
    ucb = np.minimum(qbar * diag / (diag + lam_new), 1.)
    U = np.asarray(random_state.rand(n)) <= ucb
    u = U.sum()
    assert u > 0, ('No point selected during uniform sampling step, try to increase qbar. '
                   'Expected number of points: {:.3f}'.format(n * ucb))
    X_U = X[U, :]
    # taus are RLS
    tau = compute_tau(centers_dict, X_U, similarity_func, lam_new, force_cpu)
    # RLS should always be smaller than 1
    tau = np.minimum(tau, 1.)
    # same as before, oversample by a qbar factor
    probs = np.minimum(qbar * tau, ucb[U]) / ucb[U]
    assert np.all(probs >= 0.), ('Some estimated probability is negative, this should never happen. '
                                 'Min prob: {:.5f}'.format(np.min(probs)))
    deff_estimate = probs.sum()/qbar
    assert qbar*deff_estimate >= 1., ('Estimated deff is smaller than 1, you might want to reconsider your kernel. '
                                      'deff_estimate: {:.3f}'.format(qbar*deff_estimate))
    selected = np.asarray(random_state.rand(u)) <= probs
    s = selected.sum()
    assert s > 0, ('No point selected during RLS sampling step, try to increase qbar. '
                   'Expected number of points (qbar*deff): {:.3f}'.format(np.sum(probs)))
    D_new = CentersDictionary(idx=U.nonzero()[0][selected.nonzero()[0]],
                              X=X_U[selected, :],
                              probs=probs[selected],
                              lam=lam_new,
                              qbar=qbar)
    return D_new

def bless(X, similarity_func, lam_final=2.0, qbar=2, random_state=None, H=None, force_cpu=False, verbose=True):
    n, d = X.shape
    H = H if H is not None else np.ceil(np.log(n)).astype('int')
    if random_state is None:
        rng = np.random.RandomState()
    elif isinstance(random_state, (int, int)):
        rng = np.random.RandomState(random_state)
    elif isinstance(random_state, np.random.RandomState):
        rng = random_state
    else:
        raise ValueError('Cannot understand what you passed as a random number generator.')
    diag_norm = np.asarray(similarity_func.diag(X))
    ucb_init = qbar * diag_norm / n
    selected_init = rng.rand(n) <= ucb_init
    # force at least one sample to be selected
    selected_init[0] = 1
    D = CentersDictionary(idx=selected_init.nonzero(),
                   X=X[selected_init, :],
                   probs=np.ones(np.sum(selected_init)) * ucb_init[selected_init],
                   lam=n,
                   qbar=qbar)
    lam_sequence = list(np.geomspace(lam_final, n, H))
    # discard n from the list, we already used it to initialize
    lam_sequence.pop()
    with __get_progress_bar(total=len(lam_sequence), disable=not(verbose)) as t:
        while len(lam_sequence) > 0:
            lam_new = lam_sequence.pop()
            D = reduce_lambda(X, similarity_func, D, lam_new, rng, force_cpu=force_cpu)
            t.set_postfix(lam=int(lam_new),
                          m=len(D.probs),
                          m_expected=int(D.probs.mean()*n),
                          probs_dist=f"({D.probs.mean()/qbar:.4}, {D.probs.max()/qbar:.4}, {D.probs.min()/qbar:.4})")
            t.update()
    return D

def get_nystrom_embeddings(X, centers_dict, similarity_func, force_cpu=False):
    xp = __load_gpu_module(force_cpu)
    K_XD = xp.asarray(similarity_func(X, centers_dict.X))
    U_DD, S_DD, _ = np.linalg.svd(xp.asnumpy(similarity_func(centers_dict.X, centers_dict.X)))
    U_DD, S_root_inv_DD = __stable_invert_root(U_DD, S_DD)
    K_DD_inv_sqrt = xp.asarray(U_DD * S_root_inv_DD.T)
    return K_XD.dot(K_DD_inv_sqrt)
def get_nystrom_matrix_approx(X, centers_dict, similarity_func, force_cpu=False):
    B = get_nystrom_embeddings(X, centers_dict, similarity_func, force_cpu)
    return B.dot(B.T)
def get_nystrom_PCA(X, centers_dict, similarity_func, k=-1, force_cpu=False):
    B = get_nystrom_embeddings(X, centers_dict, similarity_func, force_cpu)
    if k > B.shape[1]:
        raise ValueError('requesting k={} principal components, but the centers dictionary can only'
                         'approximate m={} components.'.format(k, B.shape[1]))
    U, Sigma, _ = np.linalg.svd(B,
                                full_matrices=False,
                                compute_uv=True)
    return np.dot(U, np.diag(Sigma))

In [2]:
from sklearn.gaussian_process.kernels import RBF
X_test = np.random.randn(30000, 10)
r = np.random.RandomState(42)
D_test = bless(X_test, RBF(length_scale=10), 10, 10, r, 10, force_cpu=True)

9/9: lam=10 m=380 m_expected=566 probs_dist=(0.001889, 0.007065, 0.000445)      


In [10]:
from sklearn.gaussian_process.kernels import RBF
similarity_func = RBF(length_scale=10)
# Create a dataset X
X = np.random.rand(100, 20)
n, d = X.shape
diag_norm = np.asarray(similarity_func.diag(X)) # it gives the diagonal of the kernel matrix = 1
qbar=2
ucb_init = qbar * diag_norm / n
rng = np.random.RandomState(42)
selected_init = rng.rand(n) <= ucb_init
# force at least one sample to be selected
selected_init[0] = 1
D = CentersDictionary(idx=selected_init.nonzero(),
                X=X[selected_init, :],
                probs=np.ones(np.sum(selected_init)) * ucb_init[selected_init],
                lam=n,
                qbar=qbar)
lam_final = 10;H =10
lam_sequence = list(np.geomspace(lam_final, n, H))

(array([ 0, 72], dtype=int64),)

In [29]:
# Compute the estimates of all RLS using the compute_tau function
lam_new = 0.2
force_cpu = False
tau_estimates = compute_tau(D, X, similarity_func, lam_new, force_cpu)
print("Tau estimates:\n", tau_estimates)

cupy not found, defaulting to numpy
Tau estimates:
 [0.01781756 0.13935384 0.14266185 0.11074688 0.11260559 0.0886903
 0.22677102 0.16531355 0.12992787 0.17682328 0.13984242 0.12395724
 0.13961938 0.13813203 0.14040698 0.15188574 0.18981059 0.16841886
 0.14274827 0.11090436 0.14115638 0.10050757 0.14082007 0.17502418
 0.16917262 0.12885603 0.1346672  0.17618711 0.10901972 0.10625629
 0.11069173 0.13687716 0.12374111 0.17275162 0.09592449 0.10571218
 0.11415503 0.11081151 0.11795866 0.14704404 0.0849966  0.12746826
 0.13138316 0.15315141 0.14677184 0.13325706 0.14712573 0.09551545
 0.15996744 0.16137679 0.09633379 0.1022168  0.14941639 0.12413216
 0.13176019 0.12759969 0.18369668 0.14822926 0.08636498 0.09541002
 0.15641286 0.08660714 0.11272029 0.0932371  0.10120348 0.136261
 0.1151679  0.14085564 0.09996032 0.12763398 0.11688655 0.13793836
 0.01781756 0.11703629 0.17414882 0.11799593 0.17266841 0.14033381
 0.17159034 0.13366023 0.14340924 0.15101238 0.15433955 0.12965325
 0.13435802 0

In [4]:
import numpy as np
from scipy.linalg import svd

def leverage_scores_sampling(A, k):
    # compute the SVD of A
    U, s, Vt = svd(A, full_matrices=False)
    # compute the leverage scores
    leverage_scores = np.sum(U**2, axis=0)
    # normalize the scores
    p = leverage_scores / np.sum(leverage_scores)
    # sample k columns with replacement according to p
    indices = np.random.choice(A.shape[1], size=k, replace=False, p=p)
    # construct the sampled matrix
    B = A[:, indices]
    return B

# Example usage
A = np.random.rand(10, 10)
k = 5
B = leverage_scores_sampling(A, k)

In [7]:
B

array([[0.60993489, 0.92288084, 0.93574355, 0.19494487, 0.97382232],
       [0.94025987, 0.18510137, 0.55533147, 0.58196297, 0.75512147],
       [0.72093542, 0.48480624, 0.94979285, 0.9043004 , 0.13503545],
       [0.5684627 , 0.60500483, 0.95821358, 0.19115622, 0.03347619],
       [0.22509391, 0.10791163, 0.1243677 , 0.06702738, 0.97213703],
       [0.3843179 , 0.44333009, 0.44619953, 0.53157502, 0.39892993],
       [0.3495867 , 0.0983109 , 0.6739625 , 0.47256958, 0.6333704 ],
       [0.86184064, 0.47377077, 0.49502417, 0.60095499, 0.4892662 ],
       [0.24085047, 0.68571423, 0.68495285, 0.12538915, 0.36963362],
       [0.64057427, 0.28317572, 0.67757728, 0.44232227, 0.30384955]])